In [5]:
import pandas as pd
import re
from functools import partial

In [71]:
df = pd.read_parquet('./../data/parsed/kiids_parsed.parquet')

In [72]:
df

,filename,raw_text,intro,cele i polityka inwestycyjna,profil ryzyka i zysku,opłaty,wyniki osiągnięte w przeszłości,informacje praktyczne,srri
0,kiid.pdf,Kluczowe informacje dla Inwestorów \n\nNiniejs...,Kluczowe informacje dla Inwestorów \n\nNiniejs...,CELE I POLITYKA INWESTYCYJNA \n\nCelem inwest...,PROFIL RYZYKA I ZYSKU \n\nNiższe ryzyko\n\nWyż...,OPŁATY \n\nJednorazowe opłaty pobierane prz...,WYNIKI OSIĄGNIĘTE W PRZESZŁOŚCI \n\nRoczne sto...,INFORMACJE PRAKTYCZNE \n\nDepozytariuszem Subf...,-1.0
1,Investor_PPK_2035_KII_21.02.2022.pdf,Kluczowe Informacje dla Inwestorów\n\nNiniejsz...,Kluczowe Informacje dla Inwestorów\n\nNiniejsz...,Cele i polityka inwestycyjna\n\n• Celem Subfun...,Profil ryzyka i zysku\n\n1\n\n2\n\n3\n\n4\n\n5...,Opłaty związane z subfunduszem\n\nIdentyfikato...,Wyniki osiągnięte w przeszłości\n\nWyniki rocz...,Informacje praktyczne\n• Depozytariuszem Subfu...,4.0


In [12]:
paragraph = df.iloc[0,3]

In [27]:
p_marked = re.sub('\n\n', '<p>', paragraph)
p_trimmed = re.sub('\s+', ' ', p_marked)
p_fixed = re.sub('<p>', '\n', p_trimmed)

# Find funds info

In [214]:
doc_text = df.raw_text[0]

In [8]:
knf_df = pd.read_pickle('./../data/knf_funds.pkl')

In [216]:
knf_df.shape

(1390, 6)

In [228]:
knf_df.head()

,fundusz,subfundusz,indentyfikator_krajowy,nr_w_rejestrze,towarzystwo,identyfikator_krajowy_funduszu
0,ABELIA CAPITAL Fundusz Inwestycyjny Zamknięty ...,None,PLFIZ000030,1212,AgioFunds Towarzystwo Funduszy Inwestycyjnych ...,PLTFI000036
1,AD ROYAL Fundusz Inwestycyjny Zamknięty Aktywó...,None,PLFIZ000572,420,AgioFunds Towarzystwo Funduszy Inwestycyjnych ...,PLTFI000036
2,AGIO Nieruchomości Komercyjnych Fundusz Inwest...,None,PLFIZ000037,986,AgioFunds Towarzystwo Funduszy Inwestycyjnych ...,PLTFI000036
3,AGIO PLUS Fundusz Inwestycyjny Otwarty,None,PLFIO000281,1309,AgioFunds Towarzystwo Funduszy Inwestycyjnych ...,PLTFI000036
4,AGIO PLUS Fundusz Inwestycyjny Otwarty,Subfundusz AGIO Akcji PLUS,PLFIO000264,1309,AgioFunds Towarzystwo Funduszy Inwestycyjnych ...,PLTFI000036


In [218]:
is_this = df_1['subfundusz'].apply(lambda text: re.search(f'{text}' , doc_text))

In [219]:
df_1[~is_this.isna()]

,fundusz,subfundusz,indentyfikator_krajowy,nr_w_rejestrze,towarzystwo,identyfikator_krajowy
19,Millennium Fundusz Inwestycyjny Otwarty,Subfundusz Obligacji Globalnych,PLSFIO00002,182,MILLENNIUM Towarzystwo Funduszy Inwestycyjnych...,PLTFI000011
85,Allianz Strategiczny Specjalistyczny Fundusz I...,Subfundusz Obligacji,PLSFIO00089,614,Towarzystwo Funduszy Inwestycyjnych Allianz Po...,PLTFI000013


In [220]:
re.search('182', doc_text)

<re.Match object; span=(712, 715), match='182'>

In [76]:
intro_text = df.intro[1]


# Add data

In [11]:
does_not_id_exist = knf_df['nr_w_rejestrze'].apply(lambda register_id: re.search(str(register_id) , intro_text)).isna()
df_filtered = knf_df[~ does_not_id_exist]

df_filtered['nr_w_rejestrze'].iloc[0]

182

In [12]:
knf_df['fundusz'].sort_values(key=lambda x: x.str.len(), ascending=False)

9     EQUES MICHAEL STRӦM CREDITUM Fundusz Inwestycy...
9     Bonus Wierzytelności 2 Pragma Inkaso Niestanda...
10    Bonus Wierzytelności 3 Pragma Inkaso Niestanda...
23    Open Finance Wierzytelności Detalicznych Niest...
17    Getback Windykacji Platinum Niestandaryzowany ...
                            ...                        
22                     BPS Fundusz Inwestycyjny Otwarty
21                     BPS Fundusz Inwestycyjny Otwarty
19                     BPS Fundusz Inwestycyjny Otwarty
20                     BPS Fundusz Inwestycyjny Otwarty
5                         LL FIZ Aktywów Niepublicznych
Name: fundusz, Length: 1390, dtype: object

In [14]:
def retrive_register_id(knf_df: 'pd.DataFrame', intro_text: str) -> int:
    
    does_not_id_exist = knf_df['nr_w_rejestrze'].apply(lambda register_id: re.search(str(register_id) , intro_text)).isna()
    df_filtered = knf_df[~ does_not_id_exist]

    if len(df_filtered['nr_w_rejestrze'].unique()) > 1:
        return None
    
    return df_filtered['nr_w_rejestrze'].iloc[0]

In [35]:
def retrive_fund_name(knf_df: 'pd.DataFrame', intro_text: str, register_id: int = None) -> str:

    if register_id is None:
        return None

    df_filtered = knf_df[knf_df['nr_w_rejestrze'] == register_id]
    full_fund_name = df_filtered['fundusz'].unique()[0]
    
    return full_fund_name

In [16]:
def retrive_subfund_name(knf_df: 'pd.DataFrame', intro_text: str) -> str:

    does_not_name_exist = knf_df['subfundusz'].apply(lambda register_id: re.search(str(register_id) , intro_text)).isna()
    df_filtered = knf_df[~ does_not_name_exist]

    fund_name = df_filtered['subfundusz'].sort_values(key=lambda x: x.str.len(), ascending=False).iloc[0]

    return fund_name

In [30]:
def retrive_organisation_name(knf_df: 'pd.DataFrame', intro_text: str) -> str:

    does_not_name_exist = knf_df['towarzystwo'].apply(lambda register_id: re.search(str(register_id) , intro_text)).isna()
    df_filtered = knf_df[~ does_not_name_exist]

    if df_filtered.size == 0:
        return None

    fund_name = df_filtered['towarzystwo'].sort_values(key=lambda x: x.str.len(), ascending=False).iloc[0]

    return fund_name

In [37]:
df.intro.apply(lambda x : retrive_register_id(knf_df, x))

0    182
Name: intro, dtype: int64

In [36]:
df.intro.apply(lambda x : retrive_fund_name(knf_df, x, 1643))

0    Investor PPK Specjalistyczny Fundusz Inwestycy...
Name: intro, dtype: object

In [38]:
df.intro.apply(lambda x : retrive_subfund_name(knf_df, x))

0    Subfundusz Obligacji Globalnych
Name: intro, dtype: object

In [85]:
df.intro.apply(lambda x : retrive_organisation_country_id(knf_df, x))

0           None
1    PLSFIO00429
Name: intro, dtype: object

In [78]:
does_not_id_exist = (
    knf_df["indentyfikator_krajowy"]
    .apply(lambda register_id: re.search(str(register_id), intro_text))
    .isna()
)
df_filtered = knf_df[~does_not_id_exist]

In [88]:
def retrive_register_id(knf_df: 'pd.DataFrame', intro_text: str) -> int:
    
    does_not_id_exist = knf_df['nr_w_rejestrze'].apply(lambda register_id: re.search(str(register_id) , intro_text)).isna()
    df_filtered = knf_df[~ does_not_id_exist]

    if len(df_filtered['nr_w_rejestrze'].unique()) != 1:
        return None
    
    return df_filtered['nr_w_rejestrze'].iloc[0]

def retrive_fund_name(knf_df: 'pd.DataFrame', intro_text: str, register_id: int = None) -> str:

    if register_id is None:
        return None

    df_filtered = knf_df[knf_df['nr_w_rejestrze'] == register_id]
    full_fund_name = df_filtered['fundusz'].unique()[0]

    return full_fund_name
   
def retrive_subfund_name(knf_df: "pd.DataFrame", intro_text: str) -> str:

    does_not_name_exist = (
        knf_df["subfundusz"]
        .apply(lambda register_id: re.search(str(register_id), intro_text))
        .isna()
    )
    df_filtered = knf_df[~does_not_name_exist]

    if df_filtered.size == 0:
        return None

    fund_name = (
        df_filtered["subfundusz"]
        .sort_values(key=lambda x: x.str.len(), ascending=False)
        .iloc[0]
    )

    return fund_name

def retrive_fund_name(knf_df: 'pd.DataFrame', intro_text: str, register_id: int = None) -> str:

    if register_id is None:
        return None

    df_filtered = knf_df[knf_df['nr_w_rejestrze'] == register_id]
    full_organisation_name = df_filtered['towarzystwo'].unique()[0]
    
    return full_organisation_name

def retrive_fund_country_id(knf_df: 'pd.DataFrame', intro_text: str):

    does_not_id_exist = (
        knf_df["indentyfikator_krajowy"]
        .apply(lambda register_id: re.search(str(register_id), intro_text))
        .isna()
    )
    df_filtered = knf_df[~does_not_id_exist]

    if df_filtered.shape[0] != 1:
        return None

    return df_filtered['indentyfikator_krajowy'].iloc[0]

def retrive_organisation_id(knf_df: 'pd.DataFrame', intro_text: str):

    does_not_id_exist = (
        knf_df["identyfikator_krajowy_funduszu"]
        .apply(lambda register_id: re.search(str(register_id), intro_text))
        .isna()
    )
    df_filtered = knf_df[~does_not_id_exist]

    if df_filtered.shape[0] != 1:
        return None

    return df_filtered['indentyfikator_krajowy'].iloc[0]

def retive_all_info(knf_df: 'pd.DataFrame', intro_text: str, register_id: int = None) -> str:
    register_id = retrive_register_id(knf_df, intro_text)
    fund_name = retrive_fund_name(knf_df, intro_text, register_id)
    subfund_name = retrive_subfund_name(knf_df, intro_text)
    organisation_name = retrive_organisation_name(knf_df, intro_text, register_id)
    fund_country_id = retrive_fund_country_id(knf_df, intro_text)
    organisation_country_id = retrive_organisation_id(knf_df, intro_text)

    return {
        'fund_name': fund_name,
        'subfund_name': subfund_name,
        'organisation_name': organisation_name,
        'register_id': register_id,
        'fund_country_id': fund_country_id,
        'organisation_country_id': organisation_country_id
    }

In [89]:
retive_all_info(knf_df, intro_text)

{'fund_name': 'Investors Towarzystwo Funduszy Inwestycyjnych S.A.',
 'subfund_name': None,
 'organisation_name': 'Investors Towarzystwo Funduszy Inwestycyjnych S.A.',
 'register_id': 1643,
 'fund_country_id': 'PLSFIO00429',
 'organisation_country_id': None}

In [51]:
knf_df[knf_df['nr_w_rejestrze'] == 182]

,fundusz,subfundusz,indentyfikator_krajowy,nr_w_rejestrze,towarzystwo,identyfikator_krajowy_funduszu
17,Millennium Specjalistyczny Fundusz Inwestycyjn...,None,PLSFIO00240,182,MILLENNIUM Towarzystwo Funduszy Inwestycyjnych...,PLTFI000011
18,Millennium Fundusz Inwestycyjny Otwarty,Subfundusz Multistrategia,PLSFIO00001,182,MILLENNIUM Towarzystwo Funduszy Inwestycyjnych...,PLTFI000011
19,Millennium Fundusz Inwestycyjny Otwarty,Subfundusz Obligacji Globalnych,PLSFIO00002,182,MILLENNIUM Towarzystwo Funduszy Inwestycyjnych...,PLTFI000011
20,Millennium Fundusz Inwestycyjny Otwarty,Subfundusz Plan Aktywny,PLSFIO00003,182,MILLENNIUM Towarzystwo Funduszy Inwestycyjnych...,PLTFI000011
21,Millennium Fundusz Inwestycyjny Otwarty,Subfundusz Plan Spokojny,PLSFIO00282,182,MILLENNIUM Towarzystwo Funduszy Inwestycyjnych...,PLTFI000011
22,Millennium Fundusz Inwestycyjny Otwarty,Subfundusz Plan Wyważony,PLSFIO00004,182,MILLENNIUM Towarzystwo Funduszy Inwestycyjnych...,PLTFI000011
23,Millennium Fundusz Inwestycyjny Otwarty,Subfundusz Top Sectors,PLSFIO00005,182,MILLENNIUM Towarzystwo Funduszy Inwestycyjnych...,PLTFI000011


In [57]:
df.intro[0]

'Kluczowe informacje dla Inwestorów \n\nNiniejszy dokument zawiera kluczowe informacje dla Inwestorów dotyczące tego Subfunduszu. Nie są to materiały marketingowe. Dostarczenie tych informacji jest wymogiem \nprawnym mającym na  celu ułatwienie zrozumienia charakteru i ryzyka związanego  z inwestowaniem w ten  Subfundusz. Przeczytanie niniejszego  dokumentu jest zalecane \nInwestorowi, aby mógł on podjąć świadomą decyzję inwestycyjną. \n\nSubfundusz Obligacji Globalnych \n\nMillennium Specjalistyczny Fundusz Inwestycyjny Otwarty \nKategoria jednostki uczestnictwa: A (reprezentatywna dla kategorii B) \nzarządzany przez Millennium TFI S.A. należące do grupy Banku Millennium \nNr w rejestrze funduszy inwestycyjnych – 182 \n\n'

In [58]:
knf_df.head(1)

,fundusz,subfundusz,indentyfikator_krajowy,nr_w_rejestrze,towarzystwo,identyfikator_krajowy_funduszu
0,ABELIA CAPITAL Fundusz Inwestycyjny Zamknięty ...,None,PLFIZ000030,1212,AgioFunds Towarzystwo Funduszy Inwestycyjnych ...,PLTFI000036


In [60]:
knf_df[knf_df['indentyfikator_krajowy'] == 'PLSFIO00429']

,fundusz,subfundusz,indentyfikator_krajowy,nr_w_rejestrze,towarzystwo,identyfikator_krajowy_funduszu
34,Investor Parasol Fundusz Inwestycyjny Otwarty,Investor PPK 2035,PLSFIO00429,1643,Investors Towarzystwo Funduszy Inwestycyjnych ...,PLTFI000018


In [75]:
df['intro'][1]

'Kluczowe Informacje dla Inwestorów\n\nNiniejszy dokument zawiera kluczowe informacje dla inwestorów dotyczące tego subfunduszu. Nie są to materiały marketingowe. Dostarczenie tych informacji jest wymogiem\nprawnym mającym na celu ułatwienie zrozumienia charakteru i ryzyka związanego z inwestowaniem w ten subfundusz. Przeczytanie niniejszego dokumentu jest zalecane\ninwestorowi, aby mógł on podjąć świadomą decyzję inwestycyjną.\n\n PPK 2035\nINVESTOR PPK 2035\nINVESTOR\nIdentyfikator krajowy Subfunduszu: PLSFIO00429 | RFI 1643\n\nSubfundusz Investor PPK SFIO zarządzanego przez Investors TFI S.A., która jest spółką z grupy Investors Holding S.A.\n\n'